# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: José Antonio Mendoza Navarro

**Professor**: Pablo Camarillo Ramirez

In [5]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Jose-Mendoza") \
    .master("spark://324c550cb62f:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 13:30:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
### Load the employees.csv and departments.csv files into PySpark DataFrames
from team_name.spark_utils import SparkUtils

employees_schema = SparkUtils.generate_schema([("employe_id", "string"), ("employe_info", "string")])
departments_schema = SparkUtils.generate_schema([("department_id", "string"), ("department_name", "string"), ("location", "string")])

employees = spark.read \
                .schema(employees_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/employees.csv")


departments = spark.read \
                .schema(departments_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/departments.csv")


employees.printSchema()

employees.show(5, truncate=False)

departments.printSchema()

departments.show(5, truncate=False)

root
 |-- employe_id: string (nullable = true)
 |-- employe_info: string (nullable = true)



+----------+-------------------------------------------------------------------------------------------+
|employe_id|employe_info                                                                               |
+----------+-------------------------------------------------------------------------------------------+
|1         |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |
|2         |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |
|3         |{'name': 'Carrie', 'department_id': 105, 'salary': 114421.44, 'hire_date': '1998-12-10'}   |
|4         |{'name': 'Renee', 'department_id': 104, 'salary': 54688.13, 'hire_date': '1995-03-17'}     |
|5         |{'name': 'Gabriella', 'department_id': 109, 'salary': 106267.03, 'hire_date': '1995-02-09'}|
+----------+-------------------------------------------------------------------------------------------+
only showing top 5 rows

root
 |-- department_id: strin

In [9]:
### Extract the following the columns: name (string), department_id (integer), salary (double), and hire_date (date) from the employee_info column
from pyspark.sql.functions import get_json_object

employees = employees.withColumn("name", get_json_object(employees.employe_info, "$.name")) \
                        .withColumn("department_id", get_json_object(employees.employe_info, "$.department_id")) \
                        .withColumn("salary", get_json_object(employees.employe_info, "$.salary")) \
                        .withColumn("hire_date", get_json_object(employees.employe_info, "$.hire_date"))

In [10]:
### Join the employees DataFrame with the departments DataFrame on department_id to enrich the employee data with department details.
employees_departments = employees.join(departments, employees.department_id == departments.department_id, "inner")
employees_departments.show(5, truncate=False)

+----------+-------------------------------------------------------------------------------------------+---------+-------------+---------+----------+-------------+-------------------+-------------+
|employe_id|employe_info                                                                               |name     |department_id|salary   |hire_date |department_id|department_name    |location     |
+----------+-------------------------------------------------------------------------------------------+---------+-------------+---------+----------+-------------+-------------------+-------------+
|1         |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |Caitlyn  |103          |115959.78|2002-06-10|103          |Sales and Marketing|Chicago      |
|2         |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |Rachel   |104          |100820.16|2009-07-01|104          |Data Engineering   |Zapopan      |
|3        

In [ ]:
### Transformations 

from pyspark.sql.functions import when, col

employees_departments = employees_departments.withColumn(
    "salary_category",
    when(col("salary").cast("double") >= 55000, "High").otherwise("Low")
)



In [12]:
### Filter and Group

high_salary_employees = employees_departments.filter(col("salary_category") == "High")
low_salary_employees = employees_departments.filter(col("salary_category") == "Low")


avg_salary_high = high_salary_employees.groupBy("department_name") \
                                       .agg({"salary": "avg"}) \
                                       .withColumnRenamed("avg(salary)", "avg_salary")


avg_salary_low = low_salary_employees.groupBy("department_name") \
                                     .agg({"salary": "avg"}) \
                                     .withColumnRenamed("avg(salary)", "avg_salary")

avg_salary_high.select("department_name", "avg_salary").show()
avg_salary_low.select("department_name", "avg_salary").show()

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
|    Customer Service|101585.01600000002|
|               Legal|  99366.3129102167|
|        Data Science|101903.63710344829|
|          Operations|100169.65621722837|
|     Human Resources|104999.43191489363|
+--------------------+------------------+

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
|    Customer Service|42644.472021276604|
|               Legal| 41160.2661

In [13]:
### Find the Top 5 employees with highest salaries from employees categorized as "High"
### Find the Top 5 employees with highest salaries from employees categorized as "Low"

top_5_high_salary_employees = high_salary_employees.orderBy(col("salary").cast("double"), ascending=False).limit(5)
top_5_low_salary_employees = low_salary_employees.orderBy(col("salary").cast("double"), ascending=False).limit(5)

top_5_high_salary_employees.select("name", "salary", "department_name").show(truncate=False)
top_5_low_salary_employees.select("name", "salary", "department_name").show(truncate=False)


+---------+---------+-------------------------------------------+
|name     |salary   |department_name                            |
+---------+---------+-------------------------------------------+
|Gabriella|149989.73|Human Resources                            |
|Katherine|149979.3 |Human Resources                            |
|Ryan     |149963.1 |Corporate Strategy and Business Development|
|Caitlyn  |149956.54|Legal                                      |
|Mark     |149915.56|Legal                                      |
+---------+---------+-------------------------------------------+

+-----+--------+-------------------------------------------+
|name |salary  |department_name                            |
+-----+--------+-------------------------------------------+
|Linda|54993.53|Corporate Strategy and Business Development|
|Tammy|54991.71|Data Engineering                           |
|Aaron|54989.45|Finance and Accounting                     |
|Craig|54945.2 |Human Resources        

In [14]:
### Compute a new column with the years in company for each employee
### Find the list of employees with more years in company and count them.

from pyspark.sql.functions import year, current_date

employees_departments = employees_departments.withColumn(
    "years_in_company",
    year(current_date()) - year(col("hire_date").cast("date"))
)


most_years_in_company = employees_departments.orderBy(col("years_in_company").desc())


most_years_in_company_count = most_years_in_company.count()

most_years_in_company.select("name", "years_in_company", "department_name").show(truncate=False)
print(f"Number of employees with the most years in company: {most_years_in_company_count}")

+--------+----------------+------------------------+
|name    |years_in_company|department_name         |
+--------+----------------+------------------------+
|Jonathan|37              |Research and Development|
|Aaron   |37              |Finance and Accounting  |
|Jorge   |37              |Customer Service        |
|Mark    |37              |Sales and Marketing     |
|William |37              |Legal                   |
|Megan   |37              |Legal                   |
|Tristan |37              |Operations              |
|Luke    |37              |Operations              |
|Tammy   |37              |Data Science            |
|James   |37              |Finance and Accounting  |
|Raymond |37              |Data Engineering        |
|Jennifer|37              |Finance and Accounting  |
|Luke    |37              |Data Engineering        |
|Ana     |37              |Data Engineering        |
|Katelyn |37              |Data Engineering        |
|Sarah   |37              |Sales and Marketing

# <center> <img src="../midterm/Jose-Mendoza-DAG.png" alt="ITESO" width="480" height="130"> </center>